We've had occasion to discuss "symplectic" geometry way back when we discussed the meaning of the complex inner product. The name symplectic is kind of a joke: it's a translation into Greek of the Latin word "complex." Symplectic geometry is related to the phase space of classical mechanics. There, things come in pairs: one always as some kind of "position" paired to a "momentum," even if the meanings of these things become somewhat abstract. Hence, a similarity to complex numbers, which have two parts, real and imaginary. And clearly, the relationship between "symplectic" and "complex" runs deep: we've seen how in the case of the oscillator, we can interpret the complex plane as the position/momentum plane of the classical oscillator!

The point is that "symplectic" or "canonical" transformations transform positions and momenta among themselves so that you end up with another set of positions and momenta.

We can consider finite linear symplectic transformations, which correspond to even dimensional "symplectic matrices".

It's easy to generate a random $2n$ x $2n$ symplectic matrix. The dimensions have to be even since we have pairs of $p$'s and $q$'s. First, generate four random real $n$ x $n$ matrices. Then tensor each one respectively with $I, iX, iY, iZ$, where $X, Y, Z$ are the $2$ x $2$ Pauli's and $I$ is the $2$ x $2$ identity. Sum them all up, and then do the $QR$ decomposition into a unitary matrix $Q$ times an upper triangular matrix $R$. $Q$ is the desired symplectic matrix.

Such a matrix preserves the so-called "symplectic form." We'll take the symplectic form $\Omega$ to be, for example, in the $6$ x $6$ case:

$\begin{pmatrix} 
0 & -1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & -1 \\
0 & 0 & 0 & 0 & 1 & 0 \end{pmatrix}$

In other words, a block diagonal matrix with $\begin{pmatrix} 0 & -1 \\ 1 & 0 \end{pmatrix}$'s along the diagonal.

A symplectic matrix $M$ will perserve $\Omega$ in the following sense:

$\Omega = M^{T}\Omega M$

In [ ]:
import qutip as qt
import numpy as np
import scipy

def random_symplectic(n):
    R = [qt.Qobj(np.random.randn(n, n)) for i in range(4)]
    M = sum([qt.tensor(R[i], O) for i, O in \
             enumerate([qt.identity(2), 1j*qt.sigmax(), \
                        1j*qt.sigmay(), 1j*qt.sigmaz()])])
    Q, R = np.linalg.qr(M.full())
    return qt.Qobj(Q)

def omega(n):
    O = np.array([[0, -1],[1, 0]])
    return qt.Qobj(scipy.linalg.block_diag(*[O]*n))

def test(S, W):
    F = M.trans()*W*M
    return np.isclose(F.full(), W.full()).all()

n = 2
W = omega(n)
M = random_symplectic(n)
print(test(M, W))

In the quantum case, we could use a symplectic matrix to transform position and momenta operators among themselves in such a way that perserves the commutation relations.

Indeed, given some $R = \begin{pmatrix} Q_{0} \\ P_{0} \\ Q_{1} \\ P_{1} \\ \vdots \end{pmatrix}$, you can think about $\Omega$ as: $\Omega_{i,j} = tr(i[R_{i}, R_{j}])$ Naturally, this won't work unless $Q$ and $P$ are infinite dimensional and satisfying the canonical commutation relation: $[Q, P] = i$.

<hr>

Now, one could imagine transforming not the $P$'s and $Q$'s but the $a$'s and $a^{\dagger}$'s. In this case, we call the transformations "Bogoliubov transformations." Linear Bogoliubov transformations can diagonalize quadratic Hamiltonians, and were first employed in the study of condensed matter systems, where certain particles can "pair up" and become "quasiparticles," moving around in their own right. In other words, the notion of "what is a particle" can be shifted with such a transformation.

In quantum optics, one simple such transformation is the following. We have a Bogoliubov matrix paramterized by $r$:

$\begin{pmatrix} cosh(r) & -sinh(r) \\ -sinh(r) & cosh(r) \end{pmatrix} \begin{pmatrix} a \\ a^{\dagger}\end{pmatrix} = \begin{pmatrix} b \\ b^{\dagger}\end{pmatrix}$

So that:

$b = a \ cosh(r) - a^{\dagger} \ sinh(r)$

$b^{\dagger} = a^{\dagger} \ cosh(r) - a \ sinh(r)$

This same transformation can be implemented with the "single mode squeezing operator":

$S = e^{ \frac{r(a^{2} - a^{\dagger2})} {2}}$

In general, we can have a complex parameter $z$ with:

$S = e^{ \frac{\overline{z}a^{2} - za^{\dagger2}}{2}}$

If $z = re^{i\theta}$:

$\begin{pmatrix} cosh(r) & -e^{i\theta}sinh(r) \\ -e^{-i\theta}sinh(r) & cosh(r) \end{pmatrix} \begin{pmatrix} a \\ a^{\dagger}\end{pmatrix} = \begin{pmatrix} b \\ b^{\dagger}\end{pmatrix}$

The squeezing operator is useful since we can act on states, for example the vacuum state $\mid 0 \rangle$ to get the "squeezed" vacuum.

In [ ]:
import qutip as qt
import numpy as np

n = 10
a = qt.destroy(n)
r = np.random.rand()

B = np.array([[np.cosh(r), -np.sinh(r)],\
              [-np.sinh(r), np.cosh(r)]])
O = [a, a.dag()]

def apply(B, O):
    n = len(O)
    return [sum([B[i][j]*O[j]\
                for j in range(n)])\
                     for i in range(n)]

a2, adag2 = apply(B, O)
print(qt.commutator(a, a.dag()) == qt.commutator(a2, adag2))

S = (r*(a*a - a.dag()*a.dag())/2).expm()
a3, adag3 = S.dag()*a*S, S.dag()*a.dag()*S

# NOTE:
# a3 is close to a2
# adag3 is close to adag2
# but not exactly due to finite approximation

vac = qt.basis(n, 0)
squeezed_vac = S.dag()*vac

print("N on vac: %.4f" % qt.expect(a.dag()*a, vac))
print("N2 on vac: %.4f" % qt.expect(a2.dag()*a2, vac))
print("N3 on vac: %.4f" % qt.expect(a3.dag()*a3, vac))
print("-")
print("N on squeezed vac: %.4f" % qt.expect(a.dag()*a, squeezed_vac))
print("N2 on squeezed vac: %.4f" % qt.expect(a2.dag()*a2, squeezed_vac))
print("N3 on squeezed vac: %.4f" % qt.expect(a3.dag()*a3, squeezed_vac))

<hr>

We can also consider "two-mode squeezing."

Given some parameter $r$, we have a Bogoliubov transformation:

$\begin{pmatrix}
cosh(r) & 0 & 0 & sinh(r) \\
0 & cosh(r) & sinh(r) & 0 \\
0 & sinh(r) & cosh(r) & 0 \\
sinh(r) & 0 & 0 & cosh(r)
\end{pmatrix} \begin{pmatrix} a_{0} \\ a_{1} \\ a_{0}^{\dagger} \\ a_{1}^{\dagger} \end{pmatrix}$

The two mode squeezing operator is:

$S = e^{r(a_{0}a_{1} - a_{0}^{\dagger}a_{1}^{\dagger})}$

In [ ]:
import qutip as qt
import numpy as np

n = 2
a = [qt.tensor(qt.destroy(n), qt.identity(n)),\
     qt.tensor(qt.identity(n), qt.destroy(n))]

r = np.random.rand()
B = np.array([[np.cosh(r), 0, 0, np.sinh(r)],\
              [0, np.cosh(r), np.sinh(r), 0],\
              [0, np.sinh(r), np.cosh(r), 0],\
              [np.sinh(r), 0, 0, np.cosh(r)]])
O = a + [a_.dag() for a_ in a]

def apply(B, O):
    n = len(O)
    return [sum([B[i][j]*O[j]\
                for j in range(n)])\
                     for i in range(n)]
O2 = apply_bog(A, B)

S = (r*(a[0]*a[1] - a[0].dag()*a[1].dag())).expm()
O3 = [S.dag()*o*S for o in O]

vac = qt.basis(n**2, 0)
vac.dims = [[n, n],[1,1]]
squeezed_vac = S.dag()*vac

print("Na on vac: %.4f" % qt.expect(a[0].dag()*a[0], vac))
print("Nb on vac: %.4f" % qt.expect(a[1].dag()*a[1], vac))

print("N2a on vac: %.4f" % qt.expect(O2[0].dag()*O2[0], vac))
print("N2b on vac: %.4f" % qt.expect(O2[1].dag()*O2[1], vac))

print("N3a on vac: %.4f" % qt.expect(O3[0].dag()*O3[0], vac))
print("N3b on vac: %.4f" % qt.expect(O3[1].dag()*O3[1], vac))

print("-")

print("Na on squeezed vac: %.4f" % qt.expect(a[0].dag()*a[0], squeezed_vac))
print("Nb on squeezed vac: %.4f" % qt.expect(a[1].dag()*a[1], squeezed_vac))

print("N2a on squeezed vac: %.4f" % qt.expect(O2[0].dag()*O2[0], squeezed_vac))
print("N2b on squeezed vac: %.4f" % qt.expect(O2[1].dag()*O2[1], squeezed_vac))

print("N3a on squeezed vac: %.4f" % qt.expect(O3[0].dag()*O3[0], squeezed_vac))
print("N3b on squeezed vac: %.4f" % qt.expect(O3[1].dag()*O3[1], squeezed_vac))

<hr>

Another application of the "two mode squeezed vacuum" is in relativistic quantum field theory in curved space.

For example, consider the famous Unruh effect. An inertial observer is hanging around Minkowski space, not detecting any particles of some scalar field. In other words, the scalar field is in its vacuum state, without any particles. All inertial observers agree on this vacuum state, and hence: on what is meant by a particle. The vacuum state is "Lorentz invariant." But what about an accelerated observer? For example, a constantly accelerated observer. The prediction of Unruh is that such an observer would detect particles: in other words, the scalar field would not be in its vacuum state.

One can describe the situation with respect to this constantly acceleration observer with "Rindler coordinates."

<img src="img/rindler.png">

The idea is that because of their constant acceleration, the world of the accelerated observer is divided into two: they're confined to the $I$ region. There is a kind of "horizon" to their experience.

Let's consider the creation and annihilation operators of a mode of momentum $w$ of the scalar field in regions $I$ and $II$ of Rindler space. In other words, $r_{I}, r_{I}^{\dagger}$ and $r_{II}, r_{II}^{\dagger}$. Suppose our observer is moving with acceleration $a$. 

Define $r = arcsinh ((e^{ \frac{2\pi w}{a}} - 1)^{-\frac{1}{2}})$

And with it the two mode squeezing operator:

$ S = e^{ r (r_{I}r_{II} - r_{I}^{\dagger}r_{II}^{\dagger})}$

If $\mid 0 \rangle_{I}\mid 0 \rangle_{II}$ is the Rindler vacuum, then: $ S^{\dagger}\mid 0 \rangle_{I}\mid 0 \rangle_{II} = \mid 0 \rangle_{M}$ gives the Minkowski vacuum.

In [ ]:
import qutip as qt
import numpy as np

n = 2
rI = qt.tensor(qt.destroy(n), qt.identity(n))
rII = qt.tensor(qt.identity(n), qt.destroy(n))
r_vac = qt.tensor(qt.basis(n,0), qt.basis(n,0))

a = np.random.rand()
w = np.random.rand()
r = np.arcsinh(1/np.sqrt(np.exp(2*np.pi*w/acc) - 1))
SQ = (r*(rI*rII - rI.dag()*rII.dag())).expm()
m_vac = SQ.dag()*r_vac

m = qt.destroy(n**2)
m.dims = [[n,n], [n,n]]
m = SQ.dag()*m*SQ

print("N in rI on rindler vac: %.4f" % qt.expect(rI.dag()*rI, r_vac))
print("N in rII on rindler vac: %.4f" % qt.expect(rII.dag()*rII, r_vac))
print("N in M on rindler vac: %.4f" % qt.expect(m.dag()*m, r_vac))
print("-")
print("N in rI on minkowski vac: %.4f" % qt.expect(rI.dag()*rI, m_vac))
print("N in rII on minkoswki vac: %.4f" % qt.expect(rII.dag()*rII, m_vac))
print("N in M on minkowski vac: %.4f" % qt.expect(m.dag()*m, m_vac))

There is an inverse relationship: if an inertial observer would measure no particles in the mode $w$, then an accelerated observer would measure some particles. And if an accelerated obeserver measured no particles, then an inertial observer would measure some.

Moreover, the accelerated observer has to trace over anything that happens in region $II$ since no signals can reach them from there. Whereas a signal from $II$ could reach our inertial observer. 